In [1]:
import pandas as pd
from unicodedata import normalize
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
df = pd.read_csv('data/df_noticias_if.csv')

In [21]:
from string import punctuation

In [22]:
df.head()

,data,titulo,conteudo
0,14/09/2017,Campus Barbacena divulga Resultado Provisório ...,\n\n\tO Campus Barbacena divulgou o Resultado ...
1,14/09/2017,Divulgado o Edital de convocação de assembleia...,\n\n\tDivulgado o Edital de convocação de asse...
2,14/09/2017,Pesquisador da Bélgica realiza palestra no Cam...,"\n\n\tO pesquisador da Bélgica, Luc Vankrunkel..."
3,14/09/2017,Divulgada a homologação das inscrições à candi...,\n\n\tDivulgada a homologação das inscrições à...
4,14/09/2017,"Aprovado Regulamento de Eventos, Cerimonial e ...","\n\n\tO Regulamento, aprovado no dia 05 de set..."


In [23]:
stopwords = nltk.corpus.stopwords.words('portuguese')
df.head()

,data,titulo,conteudo
0,14/09/2017,Campus Barbacena divulga Resultado Provisório ...,\n\n\tO Campus Barbacena divulgou o Resultado ...
1,14/09/2017,Divulgado o Edital de convocação de assembleia...,\n\n\tDivulgado o Edital de convocação de asse...
2,14/09/2017,Pesquisador da Bélgica realiza palestra no Cam...,"\n\n\tO pesquisador da Bélgica, Luc Vankrunkel..."
3,14/09/2017,Divulgada a homologação das inscrições à candi...,\n\n\tDivulgada a homologação das inscrições à...
4,14/09/2017,"Aprovado Regulamento de Eventos, Cerimonial e ...","\n\n\tO Regulamento, aprovado no dia 05 de set..."


In [24]:
def processar_textos(df):
    df['content_processado'] = df['titulo'] + ' ' + df['conteudo']
    df['content_processado'] = df['content_processado'].apply(lambda x : ''.join([c for c in x if c not in punctuation]))
    df['content_processado'] = df['content_processado'].apply(lambda x : x.replace('\n', '').replace('\t', ''))
    df['content_processado'] = df['content_processado'].apply(lambda x : normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8'))
    df['content_processado'] = df['content_processado'].apply(lambda x : ' '.join([c for c in word_tokenize(x.lower()) if (c not in stopwords)]))

In [25]:
processar_textos(df)

In [26]:
df.head()

,data,titulo,conteudo,content_processado
0,14/09/2017,Campus Barbacena divulga Resultado Provisório ...,\n\n\tO Campus Barbacena divulgou o Resultado ...,campus barbacena divulga resultado provisorio ...
1,14/09/2017,Divulgado o Edital de convocação de assembleia...,\n\n\tDivulgado o Edital de convocação de asse...,divulgado edital convocacao assembleia centro ...
2,14/09/2017,Pesquisador da Bélgica realiza palestra no Cam...,"\n\n\tO pesquisador da Bélgica, Luc Vankrunkel...",pesquisador belgica realiza palestra campus ba...
3,14/09/2017,Divulgada a homologação das inscrições à candi...,\n\n\tDivulgada a homologação das inscrições à...,divulgada homologacao inscricoes candidatura c...
4,14/09/2017,"Aprovado Regulamento de Eventos, Cerimonial e ...","\n\n\tO Regulamento, aprovado no dia 05 de set...",aprovado regulamento eventos cerimonial protoc...


In [27]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,1),
    max_features=None,
    binary=False,
    use_idf=True
)

tfidf_matrix = vectorizer.fit_transform(df['content_processado'])

In [28]:
df_matriz = pd.DataFrame(tfidf_matrix.todense(), columns=vectorizer.get_feature_names())

In [29]:
sim = cosine_similarity(tfidf_matrix)
sim.shape

(1000, 1000)

In [31]:
title = input('Título da notícia:')
content = input('Conteúdo da notícia:')


Título da notícia:campus divulga resultado de auxílio estudantil
Conteúdo da notícia:o instituto federal do sudeste de minas gerais divulgou o resultado provisório do auxílio estudantil 2018


In [32]:
df_user_input = pd.DataFrame([['', title, content]], columns=['data' , 'titulo', 'conteudo'])
processar_textos(df_user_input)
df_with_user_input = df.append(df_user_input, ignore_index=True)

In [33]:
tfidf_matrix = vectorizer.fit_transform(df_with_user_input['content_processado'])
sim = cosine_similarity(tfidf_matrix)

### 5 notícias mais similares

In [34]:
df.loc[pd.Series(sim[-1]).sort_values(ascending=False)[1:6].index]

,data,titulo,conteudo,content_processado
402,07/02/2017,Pronatec - Campus Barbacena divulga resultado ...,\n\n\tCampus Barbacena divulgou o resultado pr...,pronatec campus barbacena divulga resultado pr...
803,25/05/2016,Programa de Assistência Estudantil: Divulgado ...,\n\n\tDivulgado o resultado provisório do Prog...,programa assistencia estudantil divulgado resu...
242,25/05/2017,HOMOLOGAÇÃO DE EDITAL DE ELEIÇÃO PARA DIRETORI...,"\n\n\tNão havendo recursos contrários, o Diret...",homologacao edital eleicao diretoria centro ac...
404,06/02/2017,Campus Barbacena divulga Edital de Assistência...,\n\n\tO Campus Barbacena divulgou o Edital de ...,campus barbacena divulga edital assistencia es...
776,08/06/2016,Campus Barbacena divulga o Resultado da Anális...,\n\n\tCampus Barbacena divulga o Resultado da ...,campus barbacena divulga resultado analise cur...
